In [4]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import layout
from bokeh.models import NumeralTickFormatter
from collections import defaultdict
import os.path

# file to save the model
output_file("results.html")
quantiles = [0.5, 0.66, 0.75, 0.8, 0.9, 0.95]
stats = defaultdict(dict)

def batched_double(iterable, n=2):
    "Batch data into lists of length n. The last batch may be shorter."
    result = []
    row = []
    col = 0
    for item in iterable:
        row.append(item)
        col += 1
        if col >= n:
            result.append(row)
            col = 0
            row = []
    if row: # last row
        result.append(row)
    return result
    

for version in ['3.8', '3.9', '3.10', '3.11']:
    df = pd.read_csv(os.path.join('results', f'results_{version}_fullstats.csv')).drop(['exception', 'start_time', 'response_length'], axis=1)
    df.round({'response_time': 2})

    groups = df.groupby(['request_type', 'name'])
    
    for name, group in groups:
        qu = group.quantile(quantiles)
        stats[' '.join(name)][version] = qu

graphs = []

for url, data in stats.items():
    graph = figure(title = f"Load Test for {url}", x_axis_label = 'Quantile', y_axis_label = 'Response time (ms)')
    colors = iter(['red', 'green', 'blue', 'purple', 'brown', 'orange'])
    for version, data in data.items():
        graph.line(quantiles, data['response_time'], legend_label=version, line_color=next(colors), line_width=2)
        q50 = data['response_time'][0.5]
    
    graph.xaxis.formatter = NumeralTickFormatter(format='0 %')
    
    graphs.append(graph)

show(layout(batched_double(graphs)))

/var/folders/j4/zngd8xzn5bj14jfr2vyygwj40000gn/T/ipykernel_72969/2804320193.py:37: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  qu = group.quantile(quantiles)
/var/folders/j4/zngd8xzn5bj14jfr2vyygwj40000gn/T/ipykernel_72969/2804320193.py:37: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  qu = group.quantile(quantiles)
/var/folders/j4/zngd8xzn5bj14jfr2vyygwj40000gn/T/ipykernel_72969/2804320193.py:37: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  qu = group.quantile(